In [1]:
from typing import Set

# read KG txt files

f = open('fb60k_graph_raw.txt')
data_kg_raw = f.readlines()
print(len(data_kg_raw))
data_kg = []
for line in data_kg_raw:
    items = line[:-1].split('\t')
    head, relation, tail = items[0], items[1], items[2]
    kg_triple = {'head': head, 'tail': tail, 'relation': relation}
    data_kg.append(kg_triple)

data_kg[0]

335350


{'head': 'm.01063t',
 'tail': 'm.01063t',
 'relation': '/location/hud_county_place/place'}

In [2]:
# read corpus json files

import json

f = open('nyt10_train_raw.json')
data_corpus = json.load(f)

f = open('nyt10_test_raw.json')
tmp = json.load(f)

data_corpus = data_corpus + tmp

In [29]:
data_corpus[0]

{'sentence': 'sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor , queens , because equipment failure , not pilot error , might have been the cause . ###END###\n',
 'head': {'word': 'queens', 'id': 'm.0ccvx'},
 'tail': {'word': 'belle_harbor', 'id': 'm.05gf08'},
 'relation': '/location/location/contains'}

In [30]:
data_corpus[0]['head'], data_corpus[0]['relation'], data_corpus[0]['tail'], len(data_corpus)

({'word': 'queens', 'id': 'm.0ccvx'},
 '/location/location/contains',
 {'word': 'belle_harbor', 'id': 'm.05gf08'},
 742536)

In [3]:
valid_relation_set = set()

rel_list = ['/people/person/nationality', '/location/location/contains', '/people/person/place_lived', 
            '/people/deceased_person/place_of_death', '/people/person/ethnicity', '/people/ethnicity/people',
            '/business/person/company', '/people/person/religion', '/location/neighborhood/neighborhood_of',
            '/business/company/founders', '/people/person/children', '/location/administrative_division/country',
            '/location/country/administrative_divisions', '/business/company/place_founded', '/location/us_county/county_seat']

for i in range(len(rel_list)):
    valid_relation_set.add(rel_list[i])
    
valid_relation_set

{'/business/company/founders',
 '/business/company/place_founded',
 '/business/person/company',
 '/location/administrative_division/country',
 '/location/country/administrative_divisions',
 '/location/location/contains',
 '/location/neighborhood/neighborhood_of',
 '/location/us_county/county_seat',
 '/people/deceased_person/place_of_death',
 '/people/ethnicity/people',
 '/people/person/children',
 '/people/person/ethnicity',
 '/people/person/nationality',
 '/people/person/place_lived',
 '/people/person/religion'}

In [4]:
from tqdm import tqdm
data_corpus_new = []
data_kg_new = []
# filter relation
for item in tqdm(data_corpus):
    # if item['relation'] in valid_relation_set:
        data_corpus_new.append(item)

for item in tqdm(data_kg):
    # if item['relation'] in valid_relation_set:
        data_kg_new.append(item)

100%|██████████| 335350/335350 [00:00<00:00, 3231554.27it/s]


In [33]:
len(data_corpus_new), len(data_kg_new)

(143493, 74035)

In [34]:
# extract entity to label from corpus
entity2label = []
for item in data_corpus_new:
    head = item['head']['id'] + '\t' + item['head']['word'] + '\n'
    tail = item['tail']['id'] + '\t' + item['tail']['word'] + '\n'
    entity2label.append(head)
    entity2label.append(tail)

entity2label = [*set(entity2label)]

entity2label_str = ''

for item in entity2label:
    entity2label_str = entity2label_str + item

e2l_file = open("../../../../dataset/FB60K-NYT10/entity2label.txt", 'w', encoding='utf-8')
print(entity2label_str, file=e2l_file)

In [35]:
origin_train = open("../../../../fb_nyt_origin_data/text/train.txt").readlines()
origin_test = open("../../../../fb_nyt_origin_data/text/test.txt").readlines()

entity2label = []

def extract_e2l(e2l, dataset):
    for data in dataset:
        its = data.split('\t')
        head_id = its[0]
        tail_id = its[1]
        head_name = its[2]
        tail_name = its[3]
        head_ = head_id + '\t' + head_name + '\n'
        tail_ = tail_id + '\t' + tail_name + '\n'
        e2l.append(head_)
        e2l.append(tail_)

extract_e2l(entity2label, origin_train)
extract_e2l(entity2label, origin_test)
entity2label = [*set(entity2label)]


In [36]:
def check_e2l_mapping(e2l, corpus, kg):
    kg_entities = []
    e2l_entities = []
    corpus_entities = []
    for it in e2l:
        e2l_entities.append(it.split('\t')[0])
    e2l_entities = [*set(e2l_entities)]
    print('size of e2l entities: ', len(e2l_entities))

    for it in corpus:
        corpus_entities.append(it['head']['id'])
        corpus_entities.append(it['tail']['id'])

    corpus_entities = [*set(corpus_entities)]
    print('size of corpus entities: ', len(corpus_entities))

    for it in kg:
        kg_entities.append(it['head'])
        kg_entities.append(it['tail'])

    kg_entities = [*set(kg_entities)]
    print('size of kg entities: ', len(kg_entities))

    outer = set()
    for it in kg_entities:
        if it not in e2l_entities:
            # print(it, 'mapping is not complete')
            outer.add(it)

    print('KG entities not included in e2l: ', len(outer))

    outer = set()
    for it in corpus_entities:
        if it not in e2l_entities:
            # print(it, 'mapping is not complete')
            outer.add(it)

    print('corpus entities not included in e2l: ', len(outer))

    entities = set(kg_entities).union(set(corpus_entities))

    return entities

kg_corpus_entities = check_e2l_mapping(entity2label, data_corpus_new, data_kg_new)

size of e2l entities:  69513
size of corpus entities:  15989
size of kg entities:  34813
KG entities not included in e2l:  0
corpus entities not included in e2l:  0


In [ ]:
entity2label_str = ''

for item in entity2label:
    if item.split('\t')[0] in kg_corpus_entities:
        entity2label_str = entity2label_str + item

e2l_file = open("../../../../dataset/FB60K-NYT10/entity2label.txt", 'w', encoding='utf-8')
print(entity2label_str, file=e2l_file)

In [8]:
import random
random.seed(5583)
# make train/test/valid data
random.shuffle(data_corpus_new)
random.shuffle(data_kg_new)

lines = entity2label_str.split('\n')

e2l_mapping = {}
for line in lines[:-1]:
    e = line.split('\t')[0]
    l = line.split('\t')[1]
    e2l_mapping[e] = l

data_train = ""
data_test = ""
data_valid = ""
all_triples_by_name = ""

count = 0

triple_set = set()

for item in tqdm(data_corpus_new):
    triple = item['head']['id'] + '\t' + item['relation'] + '\t' + item['tail']['id'] + '\n'
    # triple_by_name = item['head']['word'] + '\t' + item['relation'] + '\t' + item['tail']['word'] + '\n'
    if triple not in triple_set: 
        triple_set.add(triple)
        count += 1
        # all_triples_by_name += triple_by_name
        if count == 9:
            if item['relation'] in valid_relation_set:
                data_test += triple
            continue
        if count == 10:
            if item['relation'] in valid_relation_set:
                data_valid += triple
            count = 0
            continue
        if item['relation'] in valid_relation_set:
            data_train += triple
    else:
        continue

for item in tqdm(data_kg_new):
    triple = item['head'] + '\t' + item['relation'] + '\t' + item['tail'] + '\n'
    # triple_by_name = e2l_mapping[item['head']] + '\t' + item['relation'] + '\t' + e2l_mapping[item['tail']] + '\n'
    if triple not in triple_set:
        triple_set.add(triple)
        count += 1
        # all_triples_by_name += triple_by_name
        if count == 9:
            if item['relation'] in valid_relation_set:
                data_test += triple
            continue
        if count == 10:
            if item['relation'] in valid_relation_set:
                data_valid += triple
            count = 0
            continue
        if item['relation'] in valid_relation_set:
            data_train += triple
    else:
        continue

train = open("../train.txt", 'w', encoding='utf-8')
print(data_train, file=train)

test = open("../test.txt", 'w', encoding='utf-8')
print(data_test, file=test)

valid = open("../valid.txt", 'w', encoding='utf-8')
print(data_valid, file=valid)

# triples_by_name = open("../triples_nyt10.txt", 'w', encoding='utf-8')
# print(all_triples_by_name, file=triples_by_name)

100%|██████████| 335350/335350 [00:00<00:00, 545487.13it/s]


In [ ]:
lines = entity2label_str.split('\n')

e2l_mapping = {}
for line in lines:
    e = line.split('\t')[0]
    l = line.split('\t')[1]
    e2l_mapping[e] = l


## make tripes with words instead of ids
import random
# make train/test/valid data
random.shuffle(data_corpus_new)

data_train = ""
data_test = ""
data_valid = ""

count = 0

triple_set = set()
triple_word_set = set()

for item in data_corpus_new:
    triple = item['head']['word'] + '\t' + item['relation'] + '\t' + item['tail']['word'] + '\n'
    if triple not in triple_set:
        triple_set.add(triple)
        count += 1
        if count == 9:
            data_test += triple
            continue
        if count == 10:
            data_valid += triple
            count = 0
            continue
        data_train += triple
    else:
        continue

train = open("./train_words.txt", 'w', encoding='utf-8')
print(data_train, file=train)

test = open("./test_words.txt", 'w', encoding='utf-8')
print(data_test, file=test)

valid = open("./valid_words.txt", 'w', encoding='utf-8')
print(data_valid, file=valid)

In [10]:
def load_data(data_dir, data_type="train", reverse=False):
    with open("%s%s.txt" % (data_dir, data_type), "r") as f:
        data = f.read().strip().split("\n")
        data = [i.split('\t') for i in data]
        if reverse:
            data += [[i[2], i[1]+"_reverse", i[0]] for i in data]
    return data

In [12]:
ld = load_data('../')
['m.0d05w3', '/location/country/administrative_divisions', 'm.03h64'] in ld

True